In [76]:
"""
GUV intensity analysis

Liv Jensen, Hurley Lab
University of California, Berkeley

Date Created: 2020-06-30
Date Updated: 2020-07-11

Analyzes multicolor fluorescence images to quantify fluorescence intensity at the 
perimeter of giant unilamellar vesicles (GUVs).

------------------
Some of this code is adapted from code written by Sy Redding and the Redding Lab
(github.com/ReddingLab/smtools), which is covered by the following license:
"MIT License

Copyright (c) 2018 Redding Lab, University of California, San Francisco

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE."
----------------

The remainder of the code is under the following license:

Copyright (c) 2020 Hurley Lab, University of California, Berkeley

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE. 
"""

import numpy as np
from matplotlib import pyplot as plt
from skimage import io
from skimage.filters import threshold_otsu, median
from skimage.morphology import disk
import scipy.ndimage.filters as filters
import os
%matplotlib tk

In [ ]:
filelist = []
imfolder = "pathname"
label = "label"
if not imfolder.endswith("/"):
    imfolder += "/"
for i in os.listdir(imfolder):
    if i.endswith(".tif") and not i.startswith("."):
        filelist.append(i)
first = True
selem = disk(5)
for x in filelist:
    fname = imfolder + x
    im = io.imread(fname)
    c1 = im[:,:,:,0]
    lipid= im[:,:,:,2]
    if first == True:
        signal = np.zeros((len(lipid),len(filelist)))
    first = False
    size = 5
    
# defines local maxima in the lipid channel      
    for i in range(len(lipid)):
        im_max = filters.maximum_filter(lipid[i], size)
        im_min = filters.minimum_filter(lipid[i], size)
        im_diff = im_max - im_min
        thresh = threshold_otsu(im_diff)
        thresh = thresh/2
        bool_diff = (im_diff < thresh)

    ####################
# takes average of channel 1 pixel values that are colocalized with local maxima in the lipid channel    
        masked_c1 = c1[i].copy()
        masked_c1[bool_diff] = False
        c1av = np.average(masked_c1[masked_c1 != 0])
        
    ####################
# defines background intensity as the average of 2 background values: inside-vesicle and outside-vesicle backgrounds.
# outside-background calculated for each frame is the average of non-foreground pixel values
        backgroundout = np.average(c1[i][masked_c1 == 0]) 
    
# inside-background calculated from user input of location of a vesicle in the first frame of a timecourse,
# from which the local median pixel value is used as inside background.
        if i == 0:
            plt.imshow(masked_c1)
            insidepix = plt.ginput(1,show_clicks=True)
            c1blur = median(c1[i], selem=selem)
            backgroundin = c1blur[int(round(insidepix[0][1])),int(round(insidepix[0][0]))]
        background = (backgroundout+backgroundin)/2
        
# calculates background-subtracted signal
        signal[i,filelist.index(x)] = c1av - background

# saves data as a .csv file        
np.savetxt(imfolder+label+"data.csv", signal, delimiter=',')